# 3: Train Autoencoder Network
This notebook trains a convolutional autoencoder on the hdf5 dataset created by the syllable segmentation notebooks. 
- The trained model is then used to perform dimensionality reduction and generate novel stimuli.
- Neural net from repo in files.

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [5]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline  
import os
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import hdbscan 
from sklearn import metrics

ImportError: Traceback (most recent call last):
  File "/home/caroljli/.local/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/caroljli/.local/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/caroljli/.local/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/caroljli/anaconda3/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/caroljli/anaconda3/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow 

In [ ]:
from avgn.network_analysis.network_analysis import *
import avgn.network.convnet_model as conv
from avgn.network.training import *

## Allocate GPU

In [ ]:
gpus = [3] # set CUDA to see one GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=','.join([str(i) for i in gpus])
num_gpus = len(gpus) # number of GPUs to use
if len(gpus) < 1:
    num_gpus = 1 

In [ ]:
from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print( [x.name for x in local_device_protos if x.device_type == 'GPU'] )

## Define data parameters

In [ ]:
dims = [128, 128, 1] # first dimension of input data
batch_size = 16 # size of batches to use (per GPU)

## Load the dataset

In [ ]:
from glob import glob

In [ ]:
# bird_name = 'CAVI' 

In [ ]:
hdf_locs = glob('../../../data/bf_song_syllables/BF/*_'+str(dims[0])+'.hdf5')
hdf_locs

In [ ]:
def load_from_hdf5(hdf_locs, to_load, minsize=2500):
    """Loads content from a list of HDF5 files"""
    hdf5_content = {}
    first_item=True
    for i, folder in enumerate(hdf_locs):
        with h5py.File(folder,'r') as hf:
            print(folder, len(hf[to_load[0]].value))
            if len(hf[to_load[0]].value) < minsize: continue
            if first_item:
                for tl in to_load:
                    hdf5_content[tl] = hf[tl].value
                hdf5_content['name'] = np.repeat(list(hf.attrs.values())[0], np.shape(hf['spectrograms'].value)[0])
                first_item=False
            else:
                for tl in to_load:
                    hdf5_content[tl] = np.append(hdf5_content[tl], hf[tl].value, axis = 0)
                hdf5_content['name'] = np.append(hdf5_content['name'], np.repeat(list(hf.attrs.values())[0], np.shape(hf['spectrograms'].value)[0]))
    return hdf5_content

In [ ]:
# What information is stored in the HDF5 file
to_load = ['spectrograms', 'lengths', 'start', 'wav_file', 'syll_start_rel_wav']

In [ ]:
all_content = load_from_hdf5(hdf_locs, to_load)
num_examples = len(all_content['name'])
num_examples

In [ ]:
nex=20
for i in range(3):
    fig, ax = plt.subplots(nrows=1,ncols=nex, figsize=(nex,1))
    for i in range(nex):
        ax[i].matshow(all_content['spectrograms'][np.random.randint(len(all_content['spectrograms']))].reshape((dims[0],dims[1])),
                      cmap=plt.cm.viridis, interpolation='nearest', origin='lower')
        ax[i].axis('off')

## Split dataset into training and validation sets 

In [ ]:
val_pct = .1 # how much of the dataset to set aside for validation of reconstruction

In [ ]:
validation_set = np.random.permutation(np.arange(len(all_content['spectrograms'])))[:int(len(all_content['spectrograms'])*val_pct)]
mask = np.ones(len(all_content['spectrograms']), np.bool)
mask[validation_set] = 0
validation_syllables = all_content['spectrograms'][validation_set]
training_syllables = all_content['spectrograms'][mask]
print(len(training_syllables), len(validation_syllables))

## Define the network
- Network dimensions.
- Saving network conditions.

In [ ]:
# dimensions of network
# [depth, filter size, stride] # decoder will become inverse of encoder
encoder_dims = [
    [64, 3, 1],  # 64
    [64, 3, 2], # 64
    [128, 3, 1], # 64
    [64, 3, 2], # 32
    [128, 3, 1], # 32
    [64, 3, 2], # 16
    [128, 3, 1], # 16
    [2000, 0, 0], # 8
    [2000, 0, 0], # 8
]
decoder_dims = encoder_dims[::-1]
hidden_size = 2
latent_loss = 'distance' # Either 'None', 'distance', or 'VAE'

In [ ]:
# Generate a unique key (e.g. datetime) for this training instance
network_identifier = 'Distance_AE_'+bird_name
now_string = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # this is used to identify this training instance
print(now_string)

In [6]:
import pickle

In [7]:
# save params
param_loc = '../../../data/network_params/'+network_identifier+'/'
print(param_loc+now_string+'_params.pickle')
if not os.path.exists(param_loc):
                os.makedirs(param_loc) 
with open(param_loc+now_string+'_params.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([encoder_dims, decoder_dims, hdf_locs, dims, batch_size, hidden_size, validation_set, latent_loss], f)        

NameError: name 'network_identifier' is not defined

In [ ]:
model = conv.ConvAE(dims, batch_size, encoder_dims, decoder_dims, hidden_size, latent_loss=latent_loss, network_type='AE', gpus=[0], adam_eps = 1.0e-8, activation_fn=tf.nn.relu) # eps = 0.1 and lr = 1 (after lr 0.1)

## Train the model

In [ ]:
# define a training iterator over your data
iter_ = data_iterator(training_syllables,y=None,batch_size=batch_size,num_gpus=num_gpus,dims=dims)

In [ ]:
nex=16

for ii in range(5):
    example_data = iter_.__next__()[0]
    fig, ax = plt.subplots(nrows=1,ncols=nex, figsize=(nex*2,1*2))
    for i in range(nex):
        ax[i].matshow(example_data[i].reshape((dims[0],dims[1])), cmap=plt.cm.viridis, interpolation='nearest', origin='lower', vmin=0, vmax=1)
        ax[i].axis('off')
    plt.show()

### Train Single Epoch
To test; generative network in files.

In [ ]:
learning_rate = 1e-4
latent_loss_weights = 1e-1

In [ ]:
# a list of which tensors to return from the network (e.g. train_D/G are necessary to train the network, losses are useful for plots)
return_list = ['train_D', 'train_E', 'L_d', 'L_e', 'recon_loss', 'distance_loss'] 
iter_ = data_iterator(training_syllables,y=None,batch_size=batch_size,num_gpus=num_gpus,dims=dims)
validation_iter_ = data_iterator(validation_syllables,y=None,batch_size=batch_size,num_gpus=num_gpus,dims=dims)

#train_AE(model, iter_,dataset_size = len(training_syllables), validation_iter_=False, learning_rate = 1.0, return_list=return_list)
training_df, validation_df = train_AE(model, iter_,dataset_size = int(len(training_syllables)/100), validation_iter_=validation_iter_,validation_size=len(validation_syllables),
                                      learning_rate = learning_rate, return_list=return_list, latent_loss_weights=latent_loss_weights)

In [ ]:
training_df[:3]

### Train Entire Network

In [ ]:
from datetime import datetime

In [ ]:
### Parameters, etc...
num_epochs = 50 # how many epochs to train the network for
epoch = 0 # initialize epochs
save_loc = '../../../data/models/'+network_identifier + '/'+now_string+'/'
print(save_loc)
# Visualizations (these only work if you choose a 2D latent space - write a new viz function if you didn't...)
network_save_epochs = np.unique(np.logspace(0,np.log2(num_epochs),num=20, base= 2).astype('int')) # (epochs) - which epochs to save the network
network_save_epochs=network_save_epochs[network_save_epochs>50]
#network_visualize_progress = np.unique(np.logspace(0,np.log2(num_epochs),num=10000, base= 2).astype('int')) # how often to visualize the network (leave empty list for never)
network_visualize_progress = np.arange(num_epochs)
img_save_loc = '../../../img/'+network_identifier + '/'+now_string+'/'

In [ ]:
#model.load_network('../../../../data/models/Distance_AE_CAVI/2018-10-17_10-00-27/28_model.tfmod')

In [ ]:
try:
    for epoch in tqdm(range(epoch,num_epochs)):
        
        
        # visualization
        if epoch in network_visualize_progress:
            clear_output()
            print(epoch)
            visualize_2D_AE(model, training_df, validation_df, example_data, num_examples,
                            batch_size, num_gpus, dims, iter_, n_cols = 4, std_to_plot = 2.5,
                            save_loc = img_save_loc+now_string+'/'+str(epoch)+'.jpg')
        
        # training
        iter_ = data_iterator(training_syllables,y=None,batch_size=batch_size,num_gpus=num_gpus,dims=dims)
        validation_iter_ = data_iterator(validation_syllables,y=None,batch_size=batch_size,num_gpus=num_gpus,dims=dims)
        training_df_epoch, validation_df_epoch = train_AE(model, iter_, dataset_size = len(training_syllables),
                                              validation_iter_=validation_iter_,validation_size=len(validation_syllables),
                                              learning_rate = learning_rate, return_list=return_list,
                                                         latent_loss_weights=latent_loss_weights)
        training_df = pd.concat([training_df, training_df_epoch])
        validation_df = pd.concat([validation_df, validation_df_epoch])
        
        # save network
        if epoch in network_visualize_progress:
            if not os.path.exists(save_loc):
                os.makedirs(save_loc) 
            model.save_network(save_loc+str(epoch)+'_model.tfmod')
        
except KeyboardInterrupt:
    print('interrupted by keyboard')

In [ ]:
print(epoch)

In [ ]:
### save this model
if not os.path.exists(save_loc+'manual/'):
    os.makedirs(save_loc+'manual/') 
model.save_network(save_loc+'manual/manual_model.tfmod')

### Translate Syllables in Latent Space
- Encodes and decodes syllables in neural net.

In [ ]:
def encode_x(x, z_shape,batch_size):
    nex =np.ceil(len(x)/batch_size).astype('int')
    face_z = np.zeros([nex*batch_size] + list(z_shape))
    face_x = np.zeros([nex*batch_size] + list(np.shape(x)[1:]))
    face_x[:len(x)] = x
    for batch in np.arange(nex):
        cur_batch = face_x[int(batch*batch_size):int((batch+1)*batch_size)]
        z_out = model.sess.run(model.z_x,{model.x_input: cur_batch})
        face_z[batch*batch_size:(batch+1)*batch_size,:] = z_out
    z_final = face_z[:len(x)]
    return z_final

def decode_z(z, x_shape, batch_size):
    nex =np.ceil(len(z)/batch_size).astype('int')
    face_x = np.zeros([nex*batch_size] + list(x_shape))
    face_z = np.zeros([nex*batch_size] + list(np.shape(z)[1:]))
    face_z[:len(z)] = z
    for batch in np.arange(nex):
        cur_batch = face_z[int(batch*batch_size):int((batch+1)*batch_size)]
        x_out = model.sess.run(model.x_tilde,{model.z_x: cur_batch})
        face_x[batch*batch_size:(batch+1)*batch_size,:] = x_out
    x_final = face_x[:len(z)]
    return x_final

In [ ]:
x = all_content['spectrograms']/255.
z = encode_x(np.reshape(x, (len(x), np.prod(np.shape(x)[1:]))), [hidden_size], model.batch_size)
print(np.shape(z))

In [ ]:
BirdData = pd.DataFrame({
        'specs':all_content['spectrograms'].tolist(), 
        'z':z.tolist(),
        'syllable_time': [datetime.strptime(i[0], '%d/%m/%y %H:%M:%S.%f') for i in all_content['start'].astype('str').tolist()] , 
        'syll_length_s': all_content['lengths'].tolist(), 
        'start_time_rel_wav': all_content['syll_start_rel_wav'].tolist(), 
        'original_wav': all_content['wav_file'].tolist(), 
    })
BirdData[:3]

### Generate Vizualization Grid

In [ ]:
np.min(z[:,0]), np.max(z[:,0]), np.min(z[:,1]), np.max(z[:,1])

In [ ]:
spacing = 5
xs = [-25, 25]
ys = [-125,-75]
xv,yv = draw_grid(model,dims,batch_size,xs,ys,spacing = spacing, zoom = 1)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(20,20))
ax.scatter(z[:,0], z[:,1], color='k', s=1)
ax.scatter(xv, yv, color='r', s=30)
#ax.axis('off')
plt.show()

### Syllable Interpolations
- Generating novel syllables training.

In [ ]:
### choose two points
# get their z values
# interpolate between those z values
# pass those z values into network (encode them)
# plot a figure of this interpolation, save a gif

In [ ]:
pt1 = 55000; pt2 = 10000
syllable_1 = all_content['spectrograms'][pt1]
syllable_2= all_content['spectrograms'][pt2]
fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(6,3))
ax[0].matshow(syllable_1, origin = 'lower'); ax[0].axis('off')
ax[1].matshow(syllable_2, origin = 'lower'); ax[1].axis('off')

In [ ]:
n_frames_per_interp = 32 # how many points in interp.
z1 = encode_x(np.array([syllable_1.flatten()/255.]), [2], batch_size)[0]
z2 = encode_x(np.array([syllable_2.flatten()/255.]), [2], batch_size)[0]
pcts = np.linspace(0,1,n_frames_per_interp+1)[:-1]
interp_z = np.array([(z1*pct) + (z2*(1.-pct)) for pct in tqdm(pcts, leave=False)])
x_interp = decode_z(interp_z, [np.prod(dims[:-1])], batch_size)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1, figsize=(10,10))
ax.plot([z1[0],z2[0]], [z1[1],z2[1]], color='red', lw=5)
ax.scatter(z[:,0], z[:,1], color='k', s=1, alpha=.2)

#ax.scatter(xv, yv, color='r', s=30)
ax.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=n_frames_per_interp, figsize=(n_frames_per_interp*3,3))
for frame in range(n_frames_per_interp):
    ax[frame].matshow(np.squeeze(x_interp[frame].reshape(dims)), origin = 'lower')
    ax[frame].axis('off')

### Recovering Audio Test
- From generated syllable.

In [ ]:
import pickle
import avgn.spectrogramming.spectrogramming as sg
import skimage.transform
import IPython.display
from PIL import Image

In [ ]:
# load dictionary with spectrogram parameters
dict_now_string = '2018-10-17_13-40-37'
dict_loc = '../../../data/parameter_dictionaries/'+dict_now_string+'_dict.pickle'
with open(dict_loc, 'rb') as f:
    hparams = pickle.load(f)
globals().update(hparams)

In [ ]:
_mel_basis = sg._build_mel_basis(hparams) # build a basis function if you are using a mel spectrogram
mel_inversion_filter = (_mel_basis.T / _mel_basis.sum(axis=1))
mel_inversion_filter =np.nan_to_num(np.transpose(mel_inversion_filter / mel_inversion_filter.sum(axis=1)[:, np.newaxis]))

In [ ]:
def rescale(X, out_min, out_max):
    return out_min + (X - np.min(X)) * ((out_max - out_min) / (np.max(X) - np.min(X)))

In [ ]:
def invertSyllableToWav(syll, dims, hparams):
    syll = np.squeeze(syll.reshape(dims))
    # reshape spectrogram 
    if hparams['mel_filter']:
        resize_shape = (int((np.shape(syll)[1]/hparams['resize_samp_fr']) * (1000/hparams['frame_shift_ms'])), hparams['num_freq_final'])
        syll = np.array(Image.fromarray(np.squeeze(syll)).resize(resize_shape, Image.ANTIALIAS))
        syll = np.dot(syll.T, mel_inversion_filter).T
    else:
        resize_shape = (int((np.shape(syll)[1]/hparams['resize_samp_fr']) * (1000/hparams['frame_shift_ms'])), hparams['num_freq'])
        syll = np.array(Image.fromarray(np.squeeze(syll)).resize(resize_shape, Image.ANTIALIAS))
    ### adding some noise tends to improve reconstruction quality
    syll = rescale(syll, .25,1) +np.reshape(np.random.rand(np.prod(np.shape(syll)))*.25, np.shape(syll))
    # invert spectrogram
    waveform = sg.inv_spectrogram(rescale(syll, .25,1), hparams)
    return waveform


In [ ]:
invert_wavs = [invertSyllableToWav(i, dims, hparams) for i in tqdm(x_interp)]

In [ ]:
# play back a sample of the song
IPython.display.Audio(data=np.concatenate(invert_wavs), rate=hparams['sample_rate'])